In [1]:
import pandas as pd
import model
import numpy as np

In [2]:
import os
hasDataSet = os.path.exists('./data/train_dataset.pkl')
print(hasDataSet)
train_dataset, val_dataset = model.load_training_data('./data/final_final_encoded_savant_2024.csv', hasDataSet=hasDataSet)

True
loading train_dataset from pickle
loading val_dataset from pickle


In [3]:
lstm_mod = model.train_model(train_dataset, val_dataset)

initting model architecture
starting training loop
Epoch 1, Train Loss: 0.8674, Val Loss: 0.8429
Epoch 2, Train Loss: 0.8360, Val Loss: 0.8264
Epoch 3, Train Loss: 0.8258, Val Loss: 0.8201
Epoch 4, Train Loss: 0.8204, Val Loss: 0.8119
Epoch 5, Train Loss: 0.8166, Val Loss: 0.8087
Epoch 6, Train Loss: 0.8142, Val Loss: 0.8107
Epoch 7, Train Loss: 0.8119, Val Loss: 0.8085
Epoch 8, Train Loss: 0.8102, Val Loss: 0.8089
Epoch 9, Train Loss: 0.8088, Val Loss: 0.8043
Epoch 10, Train Loss: 0.8070, Val Loss: 0.8079
Epoch 11, Train Loss: 0.8061, Val Loss: 0.8076
Epoch 12, Train Loss: 0.8050, Val Loss: 0.8034
Epoch 13, Train Loss: 0.8044, Val Loss: 0.8065
Epoch 14, Train Loss: 0.8036, Val Loss: 0.8066
Epoch 15, Train Loss: 0.8023, Val Loss: 0.8134
Epoch 16, Train Loss: 0.8020, Val Loss: 0.8092
Epoch 17, Train Loss: 0.8013, Val Loss: 0.8061
Epoch 18, Train Loss: 0.8012, Val Loss: 0.8071
Epoch 19, Train Loss: 0.8006, Val Loss: 0.8044
Epoch 20, Train Loss: 0.7997, Val Loss: 0.8083
Epoch 21, Train Lo

In [6]:
df = pd.read_csv('./data/final_final_encoded_savant_2024.csv')

In [7]:
# List of memory features (copy from your model)
memory_features = [
    'release_speed', 'release_pos_y',
    'pitch_type_group_BREAK', 'pitch_type_group_FAST', 'pitch_type_group_OFF',
    'description_ball', 'description_blocked_ball', 'description_called_strike', 'description_foul', 
    'description_foul_bunt', 'description_foul_tip', 'description_hit_by_pitch', 'description_hit_into_play', 
    'description_pitchout', 'description_swinging_strike', 'description_swinging_strike_blocked', 
    'type_B', 'type_S', 'bb_type_fly_ball', 'bb_type_ground_ball', 'bb_type_line_drive', 'bb_type_popup', 
    'zone_1.0', 'zone_2.0', 'zone_3.0', 'zone_4.0', 'zone_5.0', 'zone_6.0', 'zone_7.0', 'zone_8.0', 
    'zone_9.0', 'zone_11.0', 'zone_12.0', 'zone_13.0', 'zone_14.0', 'zone_nan', 'hit_location_1.0', 'hit_location_2.0', 
    'hit_location_3.0', 'hit_location_4.0', 'hit_location_5.0', 'hit_location_6.0', 'hit_location_7.0', 'hit_location_8.0', 
    'hit_location_9.0', 'hit_location_nan'
]

# Number of previous pitches to mock
num_prev_pitches = 5

# Create random data for each feature
mock_data = {}
np.random.seed(42)  # For reproducibility

# Example: assign plausible ranges for numeric features, 0/1 for one-hot/categorical
for feature in memory_features:
    if feature == 'release_speed':
        mock_data[feature] = np.random.uniform(-0.75, 100, num_prev_pitches)  # plausible pitch speeds
    elif feature == 'release_pos_y':
        mock_data[feature] = np.random.uniform(-0.7, 0.85, num_prev_pitches)   # plausible y positions
    else:
        # For one-hot/categorical features, use 0 or 1
        mock_data[feature] = np.random.randint(0, 2, num_prev_pitches).astype(float)

mock_memory = pd.DataFrame(mock_data).to_numpy()
context_features = [
    'stand_R', 'p_throws_R', 'balls', 'strikes',
    'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot_Top', 'at_bat_number', 'pitch_number',
    'home_score', 'away_score', 'bat_score', 'fld_score', 'n_thruorder_pitcher', 'n_priorpa_thisgame_player_at_bat',
    'prev_runs_scored'
]
context = df.iloc[100][context_features].astype(float).values

In [ ]:
import pickle

# Load model config/specs
with open("best_pitch_predictor_lstm_meta.pkl", "rb") as f:
    meta = pickle.load(f)

for key in meta:
    print(key)
    if (key == 'lstm_init_args'):
        for k in meta[key]:
            print('   ', k)

pitcher_to_id
batter_to_id
pitch_type_to_idx
lstm_init_args
    context_dim
    memory_dim
    num_pitchers
    num_batters
    pitcher_embed_dim
    batter_embed_dim
    lstm_hidden_dim
    num_pitch_types


In [ ]:
import pickle
import torch
import model

# Load model config/specs
with open("best_pitch_predictor_lstm_meta.pkl", "rb") as f:
    meta = pickle.load(f)

# Instantiate the model using the loaded specs
loaded_mod = model.PitchPredictorLSTM(
    context_dim=meta["lstm_init_args"]['context_dim'],
    memory_dim=meta["lstm_init_args"]['memory_dim'],
    num_pitchers=len(meta["pitcher_to_id"]),
    num_batters=len(meta["batter_to_id"]),
    pitcher_embed_dim=32,
    batter_embed_dim=32,
    lstm_hidden_dim=128,
    num_pitch_types=4
)

# Load the trained weights
loaded_mod.load_state_dict(torch.load("./models/best_pitch_predictor_lstm.pth", map_location="cpu"))
loaded_mod.eval()


PitchPredictorLSTM(
  (pitcher_embedding): Embedding(974, 32)
  (batter_embedding): Embedding(971, 32)
  (lstm): LSTM(46, 128, num_layers=2, batch_first=True, dropout=0.2)
  (context_fc): Sequential(
    (0): Linear(in_features=83, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
  )
  (combined_fc): Sequential(
    (0): Linear(in_features=192, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=4, bias=True)
  )
)

In [ ]:
# import torch

# # Load the model architecture
# lstm_mod = model.PitchPredictorLSTM(
#     context_dim=len(context_features),
#     memory_dim=len(memory_features),
#     num_pitchers=len(df['pitcher'].unique()),
#     num_batters=965, #len(df['batter'].unique()),
#     pitcher_embed_dim=32,
#     batter_embed_dim=32,
#     lstm_hidden_dim=128,
#     num_pitch_types=4
# )

# # Load the trained weights
# lstm_mod.load_state_dict(torch.load('best_pitch_predictor_lstm.pth', map_location='cpu'))
# lstm_mod.eval()


PitchPredictorLSTM(
  (pitcher_embedding): Embedding(974, 32)
  (batter_embedding): Embedding(965, 32)
  (lstm): LSTM(46, 128, num_layers=2, batch_first=True, dropout=0.2)
  (context_fc): Sequential(
    (0): Linear(in_features=83, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
  )
  (combined_fc): Sequential(
    (0): Linear(in_features=192, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=4, bias=True)
  )
)

In [66]:
probs = model.predict_next_pitch(loaded_mod, current_context=context, pitcher_id=[meta["pitcher_to_id"][506433]], 
                         batter_id=[meta["batter_to_id"][518595]], previous_pitches_in_pa=mock_memory)


In [69]:
probs

array([[4.34315294e-01, 1.09427735e-01, 4.56111133e-01, 1.45819620e-04]],
      dtype=float32)

In [70]:
meta['pitch_type_to_idx']
# Map the predicted probabilities to pitch type names using meta['pitch_type_to_idx']
# We'll invert the dict to get idx -> pitch_type
idx_to_pitch_type = {v: k for k, v in meta['pitch_type_to_idx'].items()}
pitch_probs = {idx_to_pitch_type[i]: float(probs[0, i]) for i in range(probs.shape[1])}
pitch_probs


{'FAST': 0.4343152940273285,
 'OFF': 0.10942773520946503,
 'BREAK': 0.4561111330986023,
 'OTH': 0.00014581962022930384}

In [ ]:
lstm_mod = model.train_model(train_dataset, val_dataset)

initting model architecture
starting training loop
Epoch 1, Train Loss: 0.8674, Val Loss: 0.8429
Epoch 2, Train Loss: 0.8360, Val Loss: 0.8264
Epoch 3, Train Loss: 0.8258, Val Loss: 0.8201
Epoch 4, Train Loss: 0.8204, Val Loss: 0.8119
Epoch 5, Train Loss: 0.8166, Val Loss: 0.8087
Epoch 6, Train Loss: 0.8142, Val Loss: 0.8107
Epoch 7, Train Loss: 0.8119, Val Loss: 0.8085
Epoch 8, Train Loss: 0.8102, Val Loss: 0.8089
Epoch 9, Train Loss: 0.8088, Val Loss: 0.8043
Epoch 10, Train Loss: 0.8070, Val Loss: 0.8079
Epoch 11, Train Loss: 0.8061, Val Loss: 0.8076
Epoch 12, Train Loss: 0.8050, Val Loss: 0.8034
Epoch 13, Train Loss: 0.8044, Val Loss: 0.8065
Epoch 14, Train Loss: 0.8036, Val Loss: 0.8066
Epoch 15, Train Loss: 0.8023, Val Loss: 0.8134
Epoch 16, Train Loss: 0.8020, Val Loss: 0.8092
Epoch 17, Train Loss: 0.8013, Val Loss: 0.8061
Epoch 18, Train Loss: 0.8012, Val Loss: 0.8071
Epoch 19, Train Loss: 0.8006, Val Loss: 0.8044
Epoch 20, Train Loss: 0.7997, Val Loss: 0.8083
Epoch 21, Train Lo

In [ ]:
import torch

# Load the model architecture
mock_lstm_mod = model.PitchPredictorLSTM(
    context_dim=len(context_features),
    memory_dim=len(memory_features),
    num_pitchers=len(df['pitcher'].unique()),
    num_batters=965, #len(df['batter'].unique()),
    pitcher_embed_dim=32,
    batter_embed_dim=32,
    lstm_hidden_dim=128,
    num_pitch_types=4
)

# Load the trained weights
mock_lstm_mod.load_state_dict(torch.load('best_pitch_predictor_lstm.pth', map_location='cpu'))
mock_lstm_mod.eval()